# Ноутбук для тестирования запросов в YaGPT через yandex cloud

In [44]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [49]:
import os
import sys

sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())))
sys.path

['/home/kostya03/miniconda3/envs/market_oracle/lib/python312.zip',
 '/home/kostya03/miniconda3/envs/market_oracle/lib/python3.12',
 '/home/kostya03/miniconda3/envs/market_oracle/lib/python3.12/lib-dynload',
 '',
 '/home/kostya03/miniconda3/envs/market_oracle/lib/python3.12/site-packages',
 '/home/kostya03/MIPT/MLOps/MarketOracle/help_src',
 '/home/kostya03/MIPT/MLOps/MarketOracle/help_src',
 '/home/kostya03/MIPT/MLOps/MarketOracle']

In [46]:
from dotenv import load_dotenv
import os

load_dotenv()

YA_TOKEN = os.getenv("YA_GPT_TOKEN")
YA_FOLDER_ID = os.getenv("YA_FOLDER_ID")
DEEPSEEK_API_KEY = os.getenv("DEEPSEEK_TOKEN")
PERP_TOKEN = os.getenv("PERP_TOKEN")

## YaGPT

In [3]:
from yandex_api import send_request

In [11]:
system_message = "Проанализируй следущий текст на семантический анализ и выдай краткое описание"
message = "Суть и цели трансформации OpenAI удивила всех, изменив стратегию: у компании не было чёткой бизнес-модели, и никто не ожидал такого спроса на AGI. В письме Сэма Альтмана от мая 2025 года говорится, что OpenAI не предвидела массового применения AGI в медицине, программировании и образовании. Эта ошибка в прогнозах заставила компанию пересмотреть подходы и показала важность гибкости.Переход к PBC был необходим для привлечения инвестиций и повышения эффективности, ведь, как пояснил Альтман, именно огромные финансовые запросы вынудили отказаться от модели «ограниченной прибыли», которая с 2019 года мешала привлекать серьёзные инвестиции, особенно на фоне конкурентов вроде Google с DeepMind и Anthropic, также выбравших структуру PBC.Переход к PBC означает, что коммерческое подразделение формально остаётся под контролем исходной некоммерческой организации. В письме Альтмана и заявлениях Бретта Тейлора подчёркивается: высший уровень управления сохраняется за некоммерческой структурой, даже при необходимости крупных инвестиций и смене капитала.Бретт Тейлор — президент и COO Salesforce, а также бывший CTO Facebook и соавтор Google Maps; председатель совета директоров OpenAI.Миссия OpenAI — служить человечеству, несмотря на понимание, что злоупотребления AGI неизбежны."

In [12]:
# from __future__ import annotations
# import time
# from yandex_cloud_ml_sdk import YCloudML

# messages_2 = [
#     {"role": "system", "text": "Найди ошибки в тексте и исправь их"},
#     {"role": "user", "text": "Ашипки саме сибя ни исрпвят."},
# ]


# sdk = YCloudML(
#     folder_id=YA_FOLDER_ID,
#     auth=YA_TOKEN,
# )

# model = sdk.models.completions("yandexgpt")

# # Variant 2: wait for the operation to complete using the wait method

# print("Variant 2:")

# operation = model.run_deferred(messages_2)

# result = operation.wait()
# print(result)

In [13]:
res = await send_request(message=message, system_message=system_message)

In [15]:
res.alternatives[0].text

'OpenAI изменила свою стратегию, так как не ожидала большого спроса на AGI (искусственный общий интеллект). Компания перешла к модели «управление по результатам» (PBC), чтобы привлечь инвестиции и повысить эффективность.\n\nНекоммерческая структура OpenAI сохранит высший уровень управления, даже при необходимости крупных инвестиций и смене капитала. Миссия компании остаётся неизменной — служить человечеству. При этом в OpenAI понимают, что злоупотребления AGI неизбежны.'

In [14]:
res

GPTModelResult(alternatives=(Alternative(role='assistant', text='OpenAI изменила свою стратегию, так как не ожидала большого спроса на AGI (искусственный общий интеллект). Компания перешла к модели «управление по результатам» (PBC), чтобы привлечь инвестиции и повысить эффективность.\n\nНекоммерческая структура OpenAI сохранит высший уровень управления, даже при необходимости крупных инвестиций и смене капитала. Миссия компании остаётся неизменной — служить человечеству. При этом в OpenAI понимают, что злоупотребления AGI неизбежны.', status=<AlternativeStatus.FINAL: 3>),), usage=Usage(input_text_tokens=280, completion_tokens=88, total_tokens=368), model_version='23.10.2024')

## Работа с DeepSeek

In [16]:
from openai import OpenAI
client = OpenAI(api_key=DEEPSEEK_API_KEY, base_url="https://api.deepseek.com")

<string>:0: RuntimeWarning: coroutine 'send_request' was never awaited


In [17]:
query = """Найди последние новости по компании 'ГК самолёт'\n
Какие- то их важные финансовые решения, а также информация про сектор их работы в целом
Дай краткий семантический анализ положения компании"""
query

"Найди последние новости по компании 'ГК самолёт'\n\nКакие- то их важные финансовые решения, а также информация про сектор их работы в целом\nДай краткий семантический анализ положения компании"

In [18]:
messages = [{"role": "user", "content": query}]
response = client.chat.completions.create(
    # model="deepseek-reasoner",
    model="deepseek-chat",
    messages=messages,
    web_search_options={
        "web_search_type": "general",
        "search_context_size": "low",
    },
    # web_search=True,
)

In [19]:
response

ChatCompletion(id='9c5bf30d-0b6f-4a2f-9e31-46c06808d807', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='### **Последние новости о компании "ГК Самолёт" (2024 г.)**  \n\n1. **Финансовые показатели и решения:**  \n   - В 2023 году "Самолёт" вошёл в топ-3 крупнейших застройщиков России по объёмам ввода жилья.  \n   - Компания активно привлекает финансирование через облигации. В начале 2024 года размещён новый выпуск на 10 млрд руб.  \n   - Выручка за 2023 год составила около 250 млрд руб., что на 15% выше, чем в 2022 году.  \n   - "Самолёт" продолжает участвовать в госпрограммах (например, льготная ипотека), что поддерживает спрос.  \n\n2. **Сектор деятельности:**  \n   - Основной фокус – массовое жильё эконом- и комфорт-класса в Москве, Подмосковье и регионах (Санкт-Петербург, Краснодарский край).  \n   - Компания активно развивает арендный бизнес (проект "Самолёт Плюс") и коммерческую недвижимость.  \n   - В условиях санкций "Самолё

In [20]:
response.choices[0].message

ChatCompletionMessage(content='### **Последние новости о компании "ГК Самолёт" (2024 г.)**  \n\n1. **Финансовые показатели и решения:**  \n   - В 2023 году "Самолёт" вошёл в топ-3 крупнейших застройщиков России по объёмам ввода жилья.  \n   - Компания активно привлекает финансирование через облигации. В начале 2024 года размещён новый выпуск на 10 млрд руб.  \n   - Выручка за 2023 год составила около 250 млрд руб., что на 15% выше, чем в 2022 году.  \n   - "Самолёт" продолжает участвовать в госпрограммах (например, льготная ипотека), что поддерживает спрос.  \n\n2. **Сектор деятельности:**  \n   - Основной фокус – массовое жильё эконом- и комфорт-класса в Москве, Подмосковье и регионах (Санкт-Петербург, Краснодарский край).  \n   - Компания активно развивает арендный бизнес (проект "Самолёт Плюс") и коммерческую недвижимость.  \n   - В условиях санкций "Самолёт" переориентировался на отечественные стройматериалы и технологии.  \n\n3. **Текущие проекты:**  \n   - Запуск новых ЖК в Новой

In [ ]:
# reasoning_content = response.choices[0].message.reasoning_content
# print(f"Reasoning: {reasoning_content}")
content = response.choices[0].message.content
print(f"Content: {content}")

## Perplexity

In [21]:
# PERP_TOKEN

In [27]:
# import requests

# url = "https://api.perplexity.ai/chat/completions"
# api_key = PERP_TOKEN

# # Заголовки запроса
# headers = {
#     "Authorization": f"Bearer {api_key}",
#     "Content-Type": "application/json"
# }

# # Данные запроса
# data = {
#     "model": "sonar",  # Модель с доступом в интернет
#     "messages": [
#         {"role": "user", "content": query}
#     ],
#     "temperature": 0.7,  # Контроль случайности ответа (0–1)
#     "max_tokens": 500     # Максимальная длина ответа
# }

# # Отправка запроса
# response = requests.post(url, headers=headers, json=data)

# # Обработка ответа
# if response.status_code == 200:
#     result = response.json()
#     answer = result["choices"][0]["message"]["content"]
#     print(answer)
# else:
#     print("Ошибка:", response.status_code, response.text)

In [26]:
# result["choices"][0]["message"]["content"]
# result

In [24]:
from perplexity_api import send_request

res = await send_request(message=query)

In [31]:
res['choices'][0]['message']['content']

"## Последние новости по компании 'ГК Самолёт'\n\n- **Региональная экспансия и отказ от новых проектов в Московском регионе**  \n  В мае 2025 года ГК «Самолёт» объявила о продолжении политики региональной экспансии и одновременно — о решении не запускать новые проекты в Московском регионе. Это связано с желанием компании придерживаться целевой маржинальности в рамках стратегии продаж[1].\n- **Финансовые результаты за 1 квартал 2025 года**  \n  По итогам первого квартала 2025 года компания отчиталась о росте выручки на 2,8% год к году (против +12,5% в предыдущем квартале и среднего исторического темпа +7,6%). При этом показатель EBITDA упал на 79,3% год к году (против -68,2% в предыдущем квартале и исторического темпа >200%). Чистый долг увеличился на 5,1 млрд рублей[5].\n- **Корпоративные решения**  \n  16 января 2025 года состоялось заседание совета директоров, на котором обсуждалось приобретение облигаций и другие корпоративные вопросы[4].\n\n## Сектор деятельности\n\nГК «Самолёт» ра

In [ ]:
res

{'id': 'f96240b3-1d8c-41e3-8ca7-2c93d5ea5fcb',
 'model': 'sonar',
 'created': 1746807383,
 'usage': {'prompt_tokens': 78,
  'completion_tokens': 500,
  'total_tokens': 578,
  'search_context_size': 'low'},
 'citations': ['https://erzrf.ru/news/gk-samolet-prodolzhayet-politiku-regionalnoy-ekspansii-i-snizheniya-urovnya-korporativnogo-dolga?regions=%D0%A0%D0%A4&tag=%D0%92%D0%B2%D0%BE%D0%B4+%D0%B6%D0%B8%D0%BB%D1%8C%D1%8F',
  'https://samolet.ru',
  'https://www.novostroy-m.ru/developer-offer-samolet_development/year-2025',
  'https://finviewer.ru/samolet/news/2025/',
  'https://www.tbank.ru/invest/stocks/SMLT/news/'],
 'object': 'chat.completion',
 'choices': [{'index': 0,
   'finish_reason': 'length',
   'message': {'role': 'assistant',
    'content': "## Последние новости по компании 'ГК Самолёт'\n\n- **Региональная экспансия и отказ от новых проектов в Московском регионе**  \n  В мае 2025 года ГК «Самолёт» объявила о продолжении политики региональной экспансии и одновременно — о решени

In [54]:
from help_src.tg_bot.services.advice_service import get_gpt_search_request, get_formatted__semantic_advice_with_gpt
from perplexity_api import send_request as perp_send_request

In [51]:
get_gpt_search_request("Сбер Банк")

'Найди последние новости по компании `Сбер Банк`\nКакие- то их важные финансовые решения, а также информация про сектор их работы в целом\n\nДай краткий семантический анализ положения компании\n\nОтвет дай в HTML-формате, а не в обычном Markdown-формате. Это очень важно\nИспользуй только поддерживаемые теги: <b>, <i>, <u>, <s>, <code>, <pre>, <a>\nНе используй теги <div>, <h>, <ul>, <li>'

In [52]:
queue = get_gpt_search_request("Сбер Банк")
queue

'Найди последние новости по компании `Сбер Банк`\nКакие- то их важные финансовые решения, а также информация про сектор их работы в целом\n\nДай краткий семантический анализ положения компании\n\nОтвет дай в HTML-формате, а не в обычном Markdown-формате. Это очень важно\nИспользуй только поддерживаемые теги: <b>, <i>, <u>, <s>, <code>, <pre>, <a>\nНе используй теги <div>, <h>, <ul>, <li>'

In [55]:
res = await perp_send_request(queue)

In [56]:
res

{'id': '385db94d-1109-4d93-aa5a-d5ba40ffd167',
 'model': 'sonar',
 'created': 1747080382,
 'usage': {'prompt_tokens': 154,
  'completion_tokens': 359,
  'total_tokens': 513,
  'search_context_size': 'low'},
 'citations': ['http://www.sberbank.ru/ru/sberpress/all',
  'https://www.sberbank.com/ru/investor-relations/groupresults/march_for_3months2025_results_for_investors_on_ras',
  'https://www.rbc.ru/quote/news/article/680f669a9a7947e141bb53fd',
  'https://www.sberbank.com/ru/investor-relations/groupresults/ras_reporting_for_february_for_2months2025',
  'https://www.forbes.ru/finansy/526658-pojti-ne-tak-mozet-vse-kak-sber-zakancivaet-god-i-kakie-riski-vidit-v-2025-m'],
 'object': 'chat.completion',
 'choices': [{'index': 0,
   'finish_reason': 'stop',
   'message': {'role': 'assistant',
    'content': '<b>Последние новости о Сбер Банке</b>\n\nСбер Банк продолжает демонстрировать стабильный рост в финансовом секторе. В первом квартале 2025 года чистая прибыль банка увеличилась на 11% год

In [57]:
res['choices'][0]['message']['content']

'<b>Последние новости о Сбер Банке</b>\n\nСбер Банк продолжает демонстрировать стабильный рост в финансовом секторе. В первом квартале 2025 года чистая прибыль банка увеличилась на 11% год к году, достигнув 404,5 млрд рублей. Рентабельность капитала составила 22%[2]. Банк также подтвердил намерение выплатить дивиденды за 2025 год в размере 50% прибыли при условии достижения норматива достаточности капитала не ниже 13,3%[3].\n\n<b>Сектор работы</b>\n\nСбербанк работает в сложных внешних и внутренних макроэкономических условиях, где инфляция остается выше таргета, а экономика замедляется. Рынок кредитования охлаждается, и банки сталкиваются с необходимостью выполнения требований достаточности капитала в условиях жесткого регулирования[5].\n\n<b>Семантический анализ положения компании</b>\n\nСемантический анализ новостей показывает, что Сбербанк остается ключевым игроком в финансовом секторе России, демонстрируя стабильный рост и адаптивность к меняющимся условиям. Однако, банк сталкивает

## Тест библиотека по парсингу markdown

In [32]:
import markdown

In [36]:
advice_text = """## Ответ perplexity:
## Последние новости о компании "Норильский никель"

- Производственная деятельность и инвестиции: Компания "Норильский никель" объявила о результатах производственной деятельности за первый квартал 2025 года1. Также, в 2025 году "Норникель" планирует выпускать 3 тыс. тонн кобальта ежегодно, что стало возможным после восстановления производства после пожара в 2022 году4.

- Финансовые решения: В мае 2025 года ПАО "ГМК Норильский никель" разместило дополнительный выпуск биржевых облигаций серии БО-001P-08-USD5. Это может указывать на усиление финансовых инструментов для привлечения инвестиций.

- Социальная и экологическая ответственность: Компания запустила новый формат поддержки инициатив активных горожан под названием "Люди территории"1. На Международном арктическом форуме "Норникель" подписал стратегическое соглашение, направленное на улучшение жизни людей в регионе2.

## Семантический анализ положения компании

Анализируя новости, можно сделать вывод, что "Норильский никель" активно работает над восстановлением и расширением производства, особенно в области кобальта. Компания также проявляет социальную ответственность и участвует в стратегических соглашениях для улучшения жизни в регионе. Финансовые решения, такие как выпуск облигаций, могут указывать на усиление инвестиционной активности.

## Овостной фон компании

Овостной фон компании можно оценить как позитивный. Это связано с успешным восстановлением производства после пожара, расширением выпуска кобальта и активной социальной политикой."""


In [37]:
html_output = markdown.markdown(advice_text)
print(html_output)

<h2>Ответ perplexity:</h2>
<h2>Последние новости о компании "Норильский никель"</h2>
<ul>
<li>
<p>Производственная деятельность и инвестиции: Компания "Норильский никель" объявила о результатах производственной деятельности за первый квартал 2025 года1. Также, в 2025 году "Норникель" планирует выпускать 3 тыс. тонн кобальта ежегодно, что стало возможным после восстановления производства после пожара в 2022 году4.</p>
</li>
<li>
<p>Финансовые решения: В мае 2025 года ПАО "ГМК Норильский никель" разместило дополнительный выпуск биржевых облигаций серии БО-001P-08-USD5. Это может указывать на усиление финансовых инструментов для привлечения инвестиций.</p>
</li>
<li>
<p>Социальная и экологическая ответственность: Компания запустила новый формат поддержки инициатив активных горожан под названием "Люди территории"1. На Международном арктическом форуме "Норникель" подписал стратегическое соглашение, направленное на улучшение жизни людей в регионе2.</p>
</li>
</ul>
<h2>Семантический анализ по